### 决策树

经验：SVM统计学习方法比西瓜书讲得好，决策树还是西瓜书讲得更好。

但是这里为了方便还是用统计里的。（其实是因为数据少一点，少花时间一点...）

因为走基本原理，这里是基于ID3的。C4.5和CART其实都差不多，只是衡量划分好坏的标准不同。

#### 数据选自 统计学习方法
 ID | 年龄 | 有工作 | 有房子 | 信贷情况 | 类别
----|------|-------|-------|---------|-----
  1 | 青年 |  否   |  否    |  一般   |  否 
  2 | 青年 |  否   |  否    |   好    |  否 
  3 | 青年 |  是   |  否    |   好    |  是 
  4 | 青年 |  是   |  是    |  一般   |  是 
  5 | 青年 |  否   |  否    |  一般   |  否       
  6 | 中年 |  否   |  否    |  一般   |  否 
  7 | 中年 |  否   |  否    |   好    |  否 
  8 | 中年 |  是   |  是    |   好    |  是 
  9 | 中年 |  否   |  是    |  非常好 |  是 
 10 | 中年 |  否   |  是    |  非常好 |  是  
 11 | 老年 |  否   |  是    |  非常好 |  是 
 12 | 老年 |  否   |  是    |   好    |  是 
 13 | 老年 |  是   |  否    |   好    |  是 
 14 | 老年 |  是   |  否    |  非常好 |  是 
 15 | 老年 |  否   |  否    |  一般   |  否   
 
#### 数值化 
 ID | A1 | A2 | A3 | A4 | Y
----|----|----|----|----|-----
  1 | 1  |  2 |  2 |  3 |  2 
  2 | 1  |  2 |  2 |  2 |  2 
  3 | 1  |  1 |  2 |  2 |  1 
  4 | 1  |  1 |  1 |  3 |  1 
  5 | 1  |  2 |  2 |  3 |  2       
  6 | 2  |  2 |  2 |  3 |  2 
  7 | 2  |  2 |  2 |  2 |  2 
  8 | 2  |  1 |  1 |  2 |  1 
  9 | 2  |  2 |  1 |  1 |  1 
 10 | 2  |  2 |  1 |  1 |  1  
 11 | 3  |  2 |  1 |  1 |  1
 12 | 3  |  2 |  1 |  2 |  1 
 13 | 3  |  1 |  2 |  2 |  1 
 14 | 3  |  1 |  2 |  1 |  1 
 15 | 3  |  2 |  2 |  3 |  2   

In [6]:
####### 1.输入数据 #######
import numpy as np
data = np.array([[1,2,2,3,2],
                 [1,2,2,2,2],
                 [1,1,2,2,1],
                 [1,1,1,3,1],
                 [1,2,2,3,2],
                 [2,2,2,3,2],
                 [2,2,2,2,2],
                 [2,1,1,2,1],
                 [2,2,1,1,1],
                 [2,2,1,1,1],
                 [3,2,1,1,1],
                 [3,2,1,2,1],
                 [3,1,2,2,1],
                 [3,1,2,1,1],
                 [3,2,2,3,2]])


In [ ]:
####### 2.算法 #######
def cal_info_gain(i,data):
    data_col = data[:,i]
    total = len(data_col)
    cates = list(set(data_col))
    cates_num = len(cates)
    H_D = 0
    H_DA = 0
    for j in range(cates_num):
        X = data_col[data_col==cates[j]]  #提取出特征i的值为j的数据行
        p = len(X)/total
        H_D += -p*np.log(p)
        dataX =  data[data_col==cates[j],:]
        dataX = dataX[:,-1] 
        H_DA += p* (len(dataX[dataX==dataX[0]]) / len(dataX))
    g = H_D-H_DA
    return g

n = data.shape[1] - 1 #初始特征数
BFS_data = [data]#把带划分的数据集存到一个list，宽度优先遍历
tree_dot = []  #记录划分特征的顺序
tree_leef = [] #记录划分好的分类
layer_num = [1,] #记录每一层非叶子(数据)节点数，方便后面宽度优先统计

while len(data)>0:
    data_old = BFS_data[0]
    MaxInfo,MaxI = 0,0
    for i in range(n): #i代表特征种类序号
        #计算每种划分的信息增益
        info_gain = cal_info_gain(i,data_old)
        if info_gain>MaxInfo:
            MaxInfo = info_gain
            MaxI = i
    tree_dot.append(i)
    #把数据集按当前划分划分为新数据集
    cates = list(set(data[:,MaxI])) #遍历该特征不同取值
    count = 0
    for j in range(len(cates)): 
        data_t = data_old[data[:,MaxI]==cates[j]]
        if np.max(data_t[:,-1]) == np.min(data_t[:,-1]):#停止条件，该类只有一种标签
             #该类数据不用再分，同时标记其对应分类
            tree_leef.append([i,j,data_t[0,-1]])
            continue
        else:
            count+=1
            BFS_data.append(data_t)
    layer_num.append(count)  
    
print(tree_dot)
print(tree_leef)
print(layer_num)

In [ ]:
####### 3.输出 #######
#整理